## 뮨맥 압축 검색기

검색 시 어려움 중 하나는 데이터를 시스템에 수집할 때 어떤 질의를 처리해야 할지 미리 알 수 없다는 점이다

- `ContextualCompressionRetriever`은 검색된 문서를 그대로 즉시 반환되는 대신, 주어진 질의의 맥락을 사용하여 문서를 압축함으로써 관련 정보만 반환되도록 할 수 있음

In [24]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS

model = ChatOllama(
  model = "gemma3:latest",
  base_url = "http://localhost:11434",
  temperature=0.7,
)

loader = TextLoader("../data/test.txt")

text_splitter = CharacterTextSplitter(
    chunk_size = 300, 
    chunk_overlap = 0,
)

texts = loader.load_and_split(text_splitter)
texts[0].page_content

'문서 1:\n\nSemantic Search\n\n정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.\n예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.\n연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝'

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# one-shot prompting 복습
oneshot_prompt = ChatPromptTemplate.from_messages([
    ("system","너는 코딩 전문가야, question에 대답하면 돼. Let's think step by step"),
    ("human","ai를 공부하려는데 python이 좋을까 C언어가 좋을까?"),
    ("human","{question}")
])

chain = oneshot_prompt | model | StrOutputParser()
res = chain.invoke({"question":"tensorflow를 잘 다룰 줄 아는데 어떤 공부를 하는게 좋을까?"})

print(res)

AI 공부를 시작하시는군요! 좋은 질문입니다. TensorFlow를 잘 다룰 줄 아신다면, 어떤 언어를 먼저 공부해야 할지, 그리고 어떤 방향으로 공부를 해야 할지에 대한 조언을 드리겠습니다.

**1. 어떤 언어를 먼저 공부해야 할까요?**

*   **Python:** AI 분야에서 가장 널리 사용되는 언어입니다. TensorFlow, PyTorch 등 주요 AI 프레임워크가 Python 기반으로 개발되었기 때문입니다. TensorFlow를 잘 다룰 줄 아신다면 Python을 먼저 배우는 것이 훨씬 유리합니다. Python은 배우기 쉽고, 다양한 라이브러리와 도구가 풍부하여 AI 모델 개발, 데이터 분석, 시각화 등 다양한 작업을 수행하기에 적합합니다.
*   **C언어:** C언어는 시스템 프로그래밍, 임베디드 시스템 등 하드웨어 제어에 강점을 가지고 있습니다. 하지만 AI 모델 개발에는 Python만큼 널리 사용되지는 않습니다. TensorFlow를 사용하기 위해 C언어를 배우는 것은 일반적으로 필요하지 않습니다.

**결론:** TensorFlow를 잘 다룰 줄 아신다면, **Python**을 우선적으로 공부하는 것이 좋습니다.

**2. TensorFlow를 잘 다룰 줄 아는데 어떤 공부를 하는 것이 좋을까요?**

TensorFlow를 잘 다룰 줄 아신다는 것은 이미 훌륭한 기반을 갖추고 계신 것을 의미합니다. 이제 TensorFlow를 더욱 깊이 이해하고 활용하기 위한 다음 단계를 준비해야 합니다. 다음은 추천하는 학습 방향입니다.

*   **TensorFlow 핵심 개념 학습:**
    *   **텐서(Tensor) 이해:** TensorFlow의 핵심 데이터 구조인 텐서에 대한 이해는 필수입니다. 텐서의 차원, 데이터 타입, 연산 등을 정확히 이해해야 합니다.
    *   **그래프(Graph) 이해:** TensorFlow는 텐서 연산을 그래프 형태로 표현합니다. 그래프의 구조, 노드, 엣지 등에 대한 이해는 모델 개발에 매우 중요합니

In [ ]:
embeddings = OllamaEmbeddings(
  model = "chatfire/bge-m3:q8_0"
)

# ollama_embeddings를 활용하여 FAISS 벡터 저장소 생성 및 검색기 변환 
# FAISS 벡터 저장소에 당신의 texts가 벡터화된 형태로 저장 
retriever = FAISS.from_documents(texts, embeddings).as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001F927C85AB0>, search_kwargs={})

In [ ]:
# 벡터 공간에서 유사도를 측정하여 상위 N개의 가장 유사한 문서에 대한 원본 텍스트 내용 반환
docs = retriever.invoke("Embedding에 대해서 알려줘")

for doc in docs:
    print(doc.page_content[:200])

데이터 마이닝
Embedding
----------------------------------------------------------------------------------------------------
문서 2:

정의: 키워드 검색은 사용자가 입력한 키워드를 기반으로 정보를 찾는 과정입니다. 이는 대부분의 검색 엔진과 데이터베이스 시스템에서 기본적인 검색 방
문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝
Page Rank
----------------------------------------------------------------------------------------------------
문서 3:

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시


In [32]:
## LLMChainExtractor를 활용하여 생성한 DocumentCompressor를 retriever에 적용한것이 
## ContextualCompressionRetriever임.

from langchain.retrievers import ContextualCompressionRetriever
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.chains import LLMChain # LLMChainExtractor의 내부 상태 변경되어 LLMChain으로 넘겨줘야됨
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader

loader = TextLoader("../data/test.txt") # This is where test.txt content is loaded
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=10)
texts = text_splitter.split_documents(documents)

ollama_embeddings = OllamaEmbeddings(
  model = "chatfire/bge-m3:q8_0"
)

retriever = FAISS.from_documents(texts, ollama_embeddings).as_retriever()

# chat 모델 생성 
llm = ChatOllama(
  model = "gemma3:latest",
  base_url = "http://localhost:11434",
  temperature=0.7,
)
"""
OpenAI 활용 시, load_dotenv로 api_key 설정 
llm = ChatOpenAI(
    temperature = 0,
    model = "gpt-4o-mini',
    api_key = api_key
)
"""

prompt_template = ChatPromptTemplate.from_messages([
    ("system","주어진 문서에서 질문과 관련된 핵심 정보를 추출하여 제공하세요. Let's think step by step"),
    ("human","문서: {context}\n\n질문:{question}")
])

# LLMChain 생성
llm_chain_for_extractor = LLMChain(llm = llm, prompt = prompt_template)

# LLM을 기반으로 문서 내용 압축; 주어진 질의에 가장 관련성이 높은 문장이나 섹션만 추출 
compressor = LLMChainExtractor(llm_chain=llm_chain_for_extractor)
# base_retriever: 원본 문서를 가져올 기본 검색기
# base_compressor: 가져온 문서를 압축할 컴프레서
# 먼저 질문과 유사한 초기 관련 문서들을 다수 검색하여 유사도를 비교하고 컴프레서를 토해 필요한 정보만을 추출 및 요약함 
compression_retriever = ContextualCompressionRetriever(
    base_compressor = compressor,
    base_retriever = retriever, #Embeddings 활용 
)

# question으로 사용 
query = "임베딩은 무엇인가요?"
# query가 포함된 핵심 정보만 추출되어 압축된 문서들의 목록을 반환 
compressed_docs = compression_retriever.get_relevant_documents(query)
print(compressed_docs[0].page_content)

제공된 문서에는 임베딩에 대한 정의가 없습니다. 문서에는 페이지 랭크 알고리즘에 대한 설명, 예시, 그리고 관련 키워드만 포함되어 있습니다.

따라서 질문에 대한 답변을 제공할 수 없습니다.



## LLMChainFilter(LLM 기반 문서 필터링)
- 초기에 검색된 문서 중 어떤 문서를 필터링하고 어떤 문서를 반활할지 결정하기 위해 LLM체인을 사용하는 보다 단순하지만 강력한 압축기 
- `ContextualCompressionRetriever`에서 `LLMChainExtractor`은 문서에서 관련 부분만 추출
- `LLMChainFilter`는 LLM을 사용하여 각 문서가 주어진 질문에 관련성이 있는지 없는지 판단하여 관련 없는 문서를 필터링하여 제거

In [ ]:
from langchain.retrievers.document_compressors import LLMChainFilter

# 검색된 각 문서 전체가 질문에 관련성이 있는지 없는지를 판단하여 관련 없는 문서를 통째로 제거 
compressor = LLMChainFilter(llm_chain = llm_chain_for_extractor)

compression_retriever = ContextualCompressionRetriever(
    base_compressor = compressor,
    base_retriever = retriever,
)

query = "바다코끼리는 무엇인가요?"
compressed_docs = compression_retriever.get_relevant_documents(query)
print(compressed_docs)

[]


## EmbeddingsFilter

- 각각의 검색된 문서에 대해 추가적인 LLM 호출을 수행하는 것은 비용이 많이 들고 속도가 느림
- `EmbeddingsFilter`는 문서와 쿼리를 임베딩하고 쿼리와 충분히 유사한 임베딩을 가진 문서만 반환함으로써 더 저렴하고 빠름
 

In [23]:
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model = "chatfire/bge-m3:q8_0"
)

# 유사도 임곗값이 0.86인 EmbeddingsFilter 객체 생성 
embeddings_filter = EmbeddingsFilter(
    embeddings = embeddings,
    similarity_threshold = 0.86
)

compression_retriever = ContextualCompressionRetriever(
    base_compressor = embeddings_filter,
    base_retriever=retriever
)

compression_docs = compression_retriever.invoke(
    "Semantic Search에 대해서 알려줘"
)

compression_docs

[]